In [1]:
print("hello")

hello


In [2]:
import re

list_tag = [
            ['*', 'ul'],
            ['1.', 'ol class="decimal"'],
            ['A.', 'ol class="upper-alpha"'],
            ['a.', 'ol class="lower-alpha"'],
            ['I.', 'ol class="upper-roman"'],
            ['i.', 'ol class="lower-roman"'],
        ]

# function for list_parser
def list_parser(text:str):
    global list_tag
    list_table = []
    open_tag_list = []
    lines = text.split('\n')
    res = ''

    # 파싱 준비
    for list_line in lines:
        # 공백 갯수
        res_line = list_line_parser(list_line)
        list_table.append(res_line)

    # 레벨 숫자
    lvl = 0
    tgn = ''
    for tbl in list_table:
        # 같은 레벨, 같은 태그명
        if lvl == tbl['level'] and tgn == tbl['type']:
            res += f"<li>{tbl['preparsed']}</li>\n"
        # 같은 레벨, 태그명만 다를 때
        elif lvl == tbl['level'] and tgn != tbl['type']:
            # 태그 닫기
            res += f"</{tgn[0:2]}>\n"
            tgn = tbl['type']
            open_tag_list[-1] = tgn
            res += f"<{tbl['type']}>\n"
            res += f"<li>{tbl['preparsed']}</li>\n"
        # 레벨값보다 수준이 더 클 때
        elif lvl +1 == tbl['level']:
            res += f"<{tbl['type']}>\n"
            res += f"<li>{tbl['preparsed']}</li>\n"
            lvl = tbl['level']
            tgn = tbl['type']
            open_tag_list.append(tbl['type'])
        # 레벨값보다 수준이 더 작을 때
        elif lvl > tbl['level']:
            for tn in open_tag_list[:tbl['level']-1:-1]:
                res += f"</{tn[0:2]}>\n"

            open_tag_list = open_tag_list[0:tbl['level']]
            lvl = tbl['level']

            if open_tag_list[-1] == tbl['type']:
                res += f"<li>{tbl['preparsed']}</li>\n"
                tgn = tbl['type']
            else:
                res += f"</{open_tag_list[-1][0:2]}>\n"
                res += f"<{tbl['type']}>\n"
                res += f"<li>{tbl['preparsed']}</li>\n"
                tgn = tbl['type']

    # 마지막으로 남아있으면...
    for tgx in open_tag_list[::-1]:
        res += f"</{tgx[0:2]}>\n"

    return res

# parse a line
def list_line_parser(text:str):
    res = {}
    # 공백 갯수
    spacing = len(re.match(r"^(\s{1,5})", text).group(1))
    res['level'] = spacing
    if text[spacing] == "*":
        res['type'] = 'ul'
        res['preparsed'] = text[spacing+1:]
    else:
        for tg in list_tag[1:]:
            if text[spacing:spacing+2] == tg[0]:
                res['type'] = tg[1]
                res['preparsed'] = text[spacing+2:]
                break
    return res

In [4]:
list_line_parser(' * 텍스트')

{'level': 1, 'type': 'ul', 'preparsed': ' 텍스트'}

In [5]:
list_parser(''' * 첫나라
 * 두나라
  1. 두나라1
  1. 두나라2
 * 세나라''')

'<ul>\n<li> 첫나라</li>\n<li> 두나라</li>\n<li>\n<ol class="decimal">\n<li> 두나라1</li>\n<li> 두나라2</li>\n</ol>\n<li> 세나라</li>\n</ul>\n'